In [36]:
import numpy as np

## Lecture-4: Computational Graph Calculation

In [38]:
a= np.array([[.1,.5],[-.3, .8]])
b= np.array([[.2],[.4]])
print(b.ndim)
print(b.shape)
dot= np.matmul(a,b)
print(dot)

2
(2, 1)
[[0.22]
 [0.26]]


In [39]:
print(dot[0][0]**2+dot[1][0]**2)

0.11600000000000003


In [40]:
g=np.array([[.44],[.52]])
x=np.array([[.2],[.4]])
w= np.array([[.1,.5],[-.3, .8]])

print(g.shape)
print(x.shape)
print(w.shape)
xT= np.transpose(x)
wT=np.transpose(w)
print("wgb:", np.matmul(g,xT))
print("xgb:",np.matmul(wT, g))


(2, 1)
(2, 1)
(2, 2)
wgb: [[0.088 0.176]
 [0.104 0.208]]
xgb: [[-0.112]
 [ 0.636]]


### Forward and Backward Propagation 

In [41]:
import math
w = [2,-3,-3] # assume some random weights and data
x = [-1, -2]

# forward pass
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid function

# backward pass through the neuron (backpropagation)
ddot = (1 - f) * f # gradient on dot variable, using the sigmoid gradient derivation
dx = [w[0] * ddot, w[1] * ddot] # backprop into x
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot] # backprop into w
# we're done! we have the gradients on the inputs to the circuit
print(dx)
print(dw)

[0.3932238664829637, -0.5898357997244456]
[-0.19661193324148185, -0.3932238664829637, 0.19661193324148185]


### Staged computation
- f(x,y)=(x+p(y))/(p(x)+(x+y)^2)

In [42]:
x = 3 # example values
y = -4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden # done!
print(f)

1.5456448841066441


Computing the backprop pass is easy: We’ll go backwards and for every variable along the way in the forward pass (sigy, num, sigx, xpy, xpysqr, den, invden) we will have the same variable, but one that begins with a d, which will hold the gradient of the output of the circuit with respect to that variable.

In [43]:
# backprop f = num * invden
dnum = invden # gradient on numerator                             #(8)
dinvden = num                                                     #(8)
# backprop invden = 1.0 / den 
dden = (-1.0 / (den**2)) * dinvden                                #(7)
# backprop den = sigx + xpysqr
dsigx = (1) * dden                                                #(6)
dxpysqr = (1) * dden                                              #(6)
# backprop xpysqr = xpy**2
dxpy = (2 * xpy) * dxpysqr                                        #(5)
# backprop xpy = x + y
dx = (1) * dxpy                                                   #(4)
dy = (1) * dxpy                                                   #(4)
# backprop sigx = 1.0 / (1 + math.exp(-x))
dx += ((1 - sigx) * sigx) * dsigx # Notice += !! See notes below  #(3)
# backprop num = x + sigy
dx += (1) * dnum                                                  #(2)
dsigy = (1) * dnum                                                #(2)
# backprop sigy = 1.0 / (1 + math.exp(-y))
dy += ((1 - sigy) * sigy) * dsigy       
print(dy)                          #(1)
# done! phew

1.5922327514838093


#### Modularized implementation: forward / backward API

In [45]:
class MultiplyGate:
    def forward(self,x,y):
        self.x = x
        self.y = y
        z = x*y
        return z
    def backward(dz):
        dx = y * dz # [dz/dz * dL/dz]
        dy = x * dz # [dz/dy * dL/dz]
        return [dx,dy]

## Example feed-forward computation of a neural network

In [35]:
class Neuron:
    def neuron_trics(self, inputs):
        """Assume insputs are 1-D numpy array and bias is a number"""
        cell_body_sum= np.sum(inputs*self.weights)+ self.bias
        firing_rate=1/1+math.exp(-cell_body_sum) # activation function(sigmoid function)
        return firing_rate